In [1]:
import pandas as pd
import numpy as np
import os
import gc

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
example_df = pd.read_csv('initial_matrix_example.csv')
example_df = reduce_mem_usage(example_df)
example_df.head()

Memory usage of dataframe is 679.20 MB
Memory usage after optimization is: 106.13 MB
Decreased by 84.4%


,date_block_num,shop_id,item_id,item_cnt_month,city_code,item_category_id,type_code,subtype_code
0,0,2,19,0.0,0,40,11,4
1,0,2,27,1.0,0,19,5,10
2,0,2,28,0.0,0,30,8,55
3,0,2,29,0.0,0,23,5,16
4,0,2,32,0.0,0,40,11,4


In [4]:
my_df = pd.read_csv('initial_matrix.csv', usecols = ['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
                                                     'city_id', 'type_id', 'item_category_id',
                                                     'main_category_id', 'sub-category_id'])
my_df = reduce_mem_usage(my_df)
#my_df = my_df[my_df.select_dtypes(exclude = 'category').columns]
my_df.head()

Memory usage of dataframe is 764.10 MB
Memory usage after optimization is: 116.74 MB
Decreased by 84.7%


,date_block_num,shop_id,item_id,item_cnt_month,city_id,type_id,item_category_id,main_category_id,sub-category_id
0,0,2,19,0.0,0,2,40,11,6
1,0,2,27,1.0,0,2,19,5,12
2,0,2,28,0.0,0,2,30,8,53
3,0,2,29,0.0,0,2,23,5,18
4,0,2,32,0.0,0,2,40,11,6


In [5]:
gc.collect()

0

In [6]:
example_df.shape, my_df.shape

((11128004, 8), (11128004, 9))

In [7]:
# Find differents in dataframes
base_columns = ['date_block_num', 'shop_id', 'item_id', 'item_cnt_month', 'item_category_id']
differ = my_df[base_columns] - example_df[base_columns]
differ[(differ < 0).any(1)]


,date_block_num,shop_id,item_id,item_cnt_month,item_category_id
1698566,0,0,0,-1.0,0
8477023,0,0,0,-20.0,0


In [8]:
# Remove this outliers
damaged_indices = differ[(differ < 0).any(1)].index
my_df.drop(index=damaged_indices, inplace=True)
example_df.drop(index=damaged_indices, inplace=True)

In [9]:
np.all(my_df[base_columns] == example_df[base_columns])

True

## Feature generating

In [10]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [11]:
def get_mean_features(grouping_cols, dataset, lags):
    '''
    Automatically generate and add mean features with selected lags to the original dataframe
    ''' 
    df = dataset.copy(deep = True)
    group = dataset.groupby(grouping_cols).agg({'item_cnt_month': ['mean']})
    #create name for the new feature
    group.columns = [ '&'.join(subname for subname in grouping_cols) + '_avg_item_cnt']
    # add feature to dataframe
    df = pd.merge(df, group, on = grouping_cols, how = 'left')
    # generate and add  lags features
    feature_name = group.columns[0] # there is only one column.
    print(feature_name)
    df = lag_feature(df, lags, feature_name)
    df.drop([feature_name], axis=1, inplace=True)

    df.fillna(0, inplace = True)
    return df

### Target lags

In [12]:
# example_df = lag_feature(example_df, [1,2,3,6,12], 'item_cnt_month')
# example_df.head()

In [13]:
# my_df = lag_feature(my_df, [1,2,3,6,12], 'item_cnt_month')
# my_df.head()

### Mean lags

**'date_block_num'**

In [15]:
# First approach
my_df = get_mean_features(grouping_cols = ['date_block_num'], dataset = my_df, lags = [3, 12])


date_block_num_avg_item_cnt


0

In [16]:
# Second approach
group = example_df.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_avg_item_cnt' ]
group.reset_index(inplace=True)

example_df = pd.merge(example_df, group, on=['date_block_num'], how='left')
example_df['date_avg_item_cnt'] = example_df['date_avg_item_cnt'].astype(np.float16)
example_df = lag_feature(example_df, [3, 12], 'date_avg_item_cnt')
example_df.drop(['date_avg_item_cnt'], axis=1, inplace=True)

example_df.fillna(0, inplace=True)

In [17]:
my_df.head()

,date_block_num,shop_id,item_id,item_cnt_month,city_id,type_id,item_category_id,main_category_id,sub-category_id,date_block_num_avg_item_cnt_lag_3,date_block_num_avg_item_cnt_lag_12
0,0,2,19,0.0,0,2,40,11,6,0.0,0.0
1,0,2,27,1.0,0,2,19,5,12,0.0,0.0
2,0,2,28,0.0,0,2,30,8,53,0.0,0.0
3,0,2,29,0.0,0,2,23,5,18,0.0,0.0
4,0,2,32,0.0,0,2,40,11,6,0.0,0.0


In [20]:
example_df.head()

,date_block_num,shop_id,item_id,item_cnt_month,city_code,item_category_id,type_code,subtype_code,date_avg_item_cnt_lag_3,date_avg_item_cnt_lag_12
0,0,2,19,0.0,0,40,11,4,0.0,0.0
1,0,2,27,1.0,0,19,5,10,0.0,0.0
2,0,2,28,0.0,0,30,8,55,0.0,0.0
3,0,2,29,0.0,0,23,5,16,0.0,0.0
4,0,2,32,0.0,0,40,11,4,0.0,0.0


In [22]:
# C
np.all(my_df['date_block_num_avg_item_cnt_lag_12'] == example_df['date_avg_item_cnt_lag_12'])

True

In [ ]:
# ts = time.time()
# group = matrix.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
# group.columns = [ 'date_avg_item_cnt' ]
# group.reset_index(inplace=True)

# matrix = pd.merge(matrix, group, on=['date_block_num'], how='left')
# matrix['date_avg_item_cnt'] = matrix['date_avg_item_cnt'].astype(np.float16)
# matrix = lag_feature(matrix, [1], 'date_avg_item_cnt')
# matrix.drop(['date_avg_item_cnt'], axis=1, inplace=True)
# time.time() - ts

In [ ]:
# Mean features 
# 
# 'date_block_num', 'item_id'
# 'date_block_num', 'shop_id'
# 'date_block_num','shop_id'
# 'date_block_num', 'item_category_id'
# 'date_block_num', 'shop_id', 'item_category_id'


# 'date_block_num', 'shop_id', 'type_code'
# 'date_block_num', 'shop_id', 'subtype_code'
# 'date_block_num', 'city_code'
# 'date_block_num', 'item_id', 'city_code'
# 'date_block_num', 'type_code'
# 'date_block_num', 'subtype_code'